In [0]:
import pandas as pd
import numpy as np
import time
from pyspark.sql.functions import col, when, rand

# 실습 파일 준비하기

In [0]:
import gc

def generate_large_csv_in_chunks(file_path, chunk_size=1_000_000, total_rows=100_000_000):
    """
    DataFrame 없이 청크 단위로 대용량 CSV 파일을 DBFS에 직접 생성하는 함수
    
    매개변수:
    dbfs_path (str): 파일이 저장될 DBFS 경로
    chunk_size (int): 각 청크당 행 수
    total_rows (int): 생성할 총 행 수
    """
    
    # 카테고리 목록
    categories = ['A', 'B', 'C', 'D', 'E']
    
    # 시작 시간 기록
    start_time = time.time()
    
    # 파일 생성 및 헤더 작성
    with open(file_path, 'w') as f:
        f.write('category,value\n')
    
    # 청크 단위로 데이터 작성
    num_chunks = total_rows // chunk_size
    
    for i in range(num_chunks):
        chunk_start = time.time()
        print(f"청크 생성 중: {i+1}/{num_chunks}...")
        
        with open(file_path, 'a') as f:
            # 한 번에 작은 그룹으로 처리 (메모리 효율성)
            for j in range(0, chunk_size, 10000):
                rows_to_write = min(10000, chunk_size - j)
                
                # 카테고리 생성
                cat_indices = np.random.randint(0, len(categories), rows_to_write)
                # 값 생성
                values = np.random.rand(rows_to_write) * 100
                
                # 데이터를 직접 문자열로 변환하여 파일에 쓰기
                for k in range(rows_to_write):
                    f.write(f"{categories[cat_indices[k]]},{values[k]:.6f}\n")
                
                # 작은 청크 처리 후 메모리 정리
                del cat_indices, values
            
        chunk_end = time.time()
        print(f"청크 {i+1} 완료: {chunk_end - chunk_start:.2f}초 소요")
        gc.collect()  # 명시적 가비지 컬렉션
        
    total_time = time.time() - start_time
    print(f"파일 생성 완료: {file_path}")
    print(f"총 소요 시간: {total_time:.2f}초, 평균 초당 {total_rows/total_time:.2f}행 생성")

In [0]:
One_B_test = True

if One_B_test:
    # 10억개 짜리 파일 생성
    total_rows = 1_000_000_000
    file_name = 'large_data_1B.csv'
else:
    # 1억개 짜리 파일 생성
    total_rows = 100_000_000
    file_name = 'large_data_100M.csv'

file_path = f'/tmp/{file_name}'

In [0]:
generate_large_csv_in_chunks(file_path, chunk_size=1_000_000, total_rows=total_rows)

청크 생성 중: 1/100...
청크 1 완료: 1.29초 소요
청크 생성 중: 2/100...
청크 2 완료: 1.06초 소요
청크 생성 중: 3/100...
청크 3 완료: 0.91초 소요
청크 생성 중: 4/100...
청크 4 완료: 0.92초 소요
청크 생성 중: 5/100...
청크 5 완료: 1.00초 소요
청크 생성 중: 6/100...
청크 6 완료: 1.32초 소요
청크 생성 중: 7/100...
청크 7 완료: 0.92초 소요
청크 생성 중: 8/100...
청크 8 완료: 0.93초 소요
청크 생성 중: 9/100...
청크 9 완료: 0.92초 소요
청크 생성 중: 10/100...
청크 10 완료: 0.93초 소요
청크 생성 중: 11/100...
청크 11 완료: 0.91초 소요
청크 생성 중: 12/100...
청크 12 완료: 0.91초 소요
청크 생성 중: 13/100...
청크 13 완료: 1.04초 소요
청크 생성 중: 14/100...
청크 14 완료: 0.90초 소요
청크 생성 중: 15/100...
청크 15 완료: 0.93초 소요
청크 생성 중: 16/100...
청크 16 완료: 0.91초 소요
청크 생성 중: 17/100...
청크 17 완료: 0.92초 소요
청크 생성 중: 18/100...
청크 18 완료: 0.92초 소요
청크 생성 중: 19/100...
청크 19 완료: 0.97초 소요
청크 생성 중: 20/100...
청크 20 완료: 0.90초 소요
청크 생성 중: 21/100...
청크 21 완료: 0.93초 소요
청크 생성 중: 22/100...
청크 22 완료: 0.93초 소요
청크 생성 중: 23/100...
청크 23 완료: 0.95초 소요
청크 생성 중: 24/100...
청크 24 완료: 0.93초 소요
청크 생성 중: 25/100...
청크 25 완료: 0.93초 소요
청크 생성 중: 26/100...
청크 26 완료: 0.91초 소요
청크 생성 중: 27/100...
청크 27 완료: 0

In [0]:
%sh

ls -tl /tmp/*.csv

-rw-r--r-- 1 root root 11899997845 Mar 24 06:05 /tmp/large_data_1B.csv
-rw-r--r-- 1 root root  1190002162 Mar 24 05:42 /tmp/large_data_100M.csv


In [0]:
dbutils.fs.cp(f"file:{file_path}", f"dbfs:/{file_name}")

True

In [0]:
# 파일 확인 및 크기 체크
display(dbutils.fs.ls("dbfs:/"))

path,name,size,modificationTime
dbfs:/car_sales_data.csv,car_sales_data.csv,45379,1742618060000
dbfs:/databricks-datasets/,databricks-datasets/,0,0
dbfs:/databricks-results/,databricks-results/,0,0
dbfs:/large_data_100M.csv,large_data_100M.csv,1190002162,1742795067000
dbfs:/large_data_1B.csv,large_data_1B.csv,11899997845,1742768984000
dbfs:/monthly_sales_by_brand/,monthly_sales_by_brand/,0,0
dbfs:/user/,user/,0,0


In [0]:
%fs ls dbfs:/

path,name,size,modificationTime
dbfs:/car_sales_data.csv,car_sales_data.csv,45379,1742618060000
dbfs:/databricks-datasets/,databricks-datasets/,0,0
dbfs:/databricks-results/,databricks-results/,0,0
dbfs:/large_data_100M.csv,large_data_100M.csv,1190002162,1742795067000
dbfs:/large_data_1B.csv,large_data_1B.csv,11899997845,1742768984000
dbfs:/monthly_sales_by_brand/,monthly_sales_by_brand/,0,0
dbfs:/user/,user/,0,0


# Spark으로 처리해보기

In [0]:
# 실행 모드 (deploy mode) 확인
print("Deploy mode:", spark.conf.get("spark.submit.deployMode", "Not Set")) 

Deploy mode: Not Set


In [0]:
# lazy execution이 기본
# show, display, count, collect, write과 같은 action이 실행되어야 실제 코드들이 실행됨.
# - Action의 반대는 transformation (filter, groupBy, select)
df = spark.read.option("header", "true") \
               .csv(f"dbfs:/{file_name}")

In [0]:
# 하지만 inferSchema를 true로 주면 lazy execution이 무시되고 바로 실행됨
# - 원래는 샘플링을 해야하는데 Databricks Community Edition의 로컬 모드는 모든 것을 읽는 걸로 보임 
df = spark.read.option("header", "true") \
               .option("inferSchema", "true") \
               .csv(f"dbfs:/{file_name}")
               

In [0]:
display(df)  # action으로 간주됨

category,value
B,25.394258
E,86.717396
C,64.641403
B,21.997705
B,77.811007
B,18.464706
E,27.865266
D,38.638753
A,42.36043
D,23.679677


In [0]:
num_partitions = df.rdd.getNumPartitions()
print("Number of partitions:", num_partitions)

Number of partitions: 89


In [0]:
record_count = df.count()
print("Number of records:", record_count)

Number of records: 1000000000


In [0]:
from pyspark.sql.functions import avg

# 벤치마크: 카테고리별 평균 계산
print("\n카테고리별 평균 계산:")

spark_start = time.time()
spark_result = df.groupby('category').agg(avg('value').alias('value_avg')).orderBy('category')
spark_result_collected = spark_result.collect()  # 액션 실행
spark_time = time.time() - spark_start
print(f"Spark 실행 시간: {spark_time:.2f}초")
print(spark_result_collected)


카테고리별 평균 계산:
Spark 실행 시간: 886.97초
[Row(category='A', value_avg=50.00028650266013), Row(category='B', value_avg=50.00206989217908), Row(category='C', value_avg=50.00158353411273), Row(category='D', value_avg=50.00117236714155), Row(category='E', value_avg=50.00116377910072)]


# Pandas로 처리해보기

In [0]:
pandas_df = pd.read_csv(file_path)
pandas_df.head()

---------------------------------------------------------------------------
The Python process exited with exit code 137 (SIGKILL: Killed). This may have been caused by an OOM error. Check your command's memory usage.



The last 10 KB of the process's stderr and stdout can be found below. See driver logs for full logs.
---------------------------------------------------------------------------
Last messages on stderr:
Mon Mar 24 05:33:54 2025 Connection to spark from PID  954
Mon Mar 24 05:33:54 2025 Initialized gateway on port 45569
Mon Mar 24 05:33:55 2025 Connected to spark.
---------------------------------------------------------------------------
Last messages on stdout:
NOTE: When using the `ipython kernel` entry point, Ctrl-C will not work.

To exit, you will have to explicitly quit this process, by either sending
"quit" from a client, or using Ctrl-\ in UNIX-like environments.

To read more about this, see https://github.com/ipython/ipython/issues/2049


To connect another cl

In [0]:
# 벤치마크: 카테고리별 평균 계산
print("\n카테고리별 평균 계산:")

start_pandas = time.time()    
# pandas_counts = pandas_df['category'].value_counts().sort_index()
pandas_counts = pandas_df.groupby('category', as_index=False)['value'].mean().sort_values('category')
end_pandas = time.time()    
print(f"Pandas Execution Time: {end_pandas - start_pandas:.2f} seconds")

print(pandas_counts.head())


카테고리별 평균 계산:
Pandas Execution Time: 7.46 seconds
  category      value
0        A  50.007882
1        B  49.995315
2        C  50.012145
3        D  50.007900
4        E  49.994069
